In [1]:
import pandas as pd
import requests
import numpy as np

In [2]:
df = pd.read_csv('../../raw_data/predict_set.csv',usecols=['date','Company_name','text','full_text','storyId'],index_col='date',parse_dates=True).squeeze()

In [3]:
df['full_text'] = df['full_text'].astype(str)
df['text'] = df['text'].astype(str)
df['storyId'] = df['storyId'].astype(str)
df['Company_name'] = df['Company_name'].astype(str)


In [4]:
df.loc[df.index[0],'full_text'].values[0]

'\n\n\n          ENR.GOFIM.554.1\n        \n\n          Date: 22/12/2022\n        \n\n          Notice N° AV-2022-148\n        \n\n          (Only the french version prevails)\n        \n\n\n\n\n                Event\n              \n\n\n\n                Change in coupon rate\n              \n\n\n\n\n\n                Involved Instrument(s)\n              \n\n\n\n                28DEC2017 2.22% 7A 100K ATW\n              \n\n\n\n\n\n\n\n\n\n\n          - NOTICE SUBJECT\n        \n\n          Change in coupon rate of "ATTIJARIWAFA BANK" bonds of 28/12/2017\n        \n\n          - Change in coupon rate\n        \n\n\n\n\n                Frequency change\n              \n\n\n\n                Annual\n              \n\n\n\n\n\n\n\n\n\n\n\n                Securitie Ticker\n              \n\n\n\n                OATWQ\n              \n\n\n\n\n\n\n\n\n\n\n\n                Tranche\n              \n\n\n\n                B\n              \n\n\n\n\n\n\n\n\n\n\n\n                Due date\n      

In [5]:
for i in range(len(df)) :
    try:
        process_date = np.datetime64('now')
        company_response_get = requests.get(f"http://127.0.0.1:8000/companies/{df.loc[df.index[i],'Company_name'].values[0]}")
        if company_response_get.status_code == 404:
            company_response = requests.post("http://127.0.0.1:8000/company/add/",json={'name':df.loc[df.index[i],'Company_name'].values[0],
                                                                                    'description':'No description provided'})

        model_response = requests.post("http://127.0.0.1:8000/article/predict/",json={'body':df.loc[df.index[i],'full_text'].values[0]})
        response = requests.post(f"http://127.0.0.1:8000/article/add/{df.loc[df.index[i],'Company_name'].values[0]}",json={
            "date": df.index[i].strftime('%Y-%m-%d'),
            "title": df.loc[df.index[i],'text'].values[0],
            "uploaded_at": pd.to_datetime(str(process_date)).strftime('%Y-%m-%d'),
            "body": df.loc[df.index[i],'full_text'].values[0],
            "sourceURL": df.loc[df.index[i],'storyId'].values[0],
            "topic_category": model_response.json()['topic_category'],
            "esg_score": model_response.json()['esg_score'],
            "scored_at": pd.to_datetime(str(process_date)).strftime('%Y-%m-%d'),
            "exclude_count": 0
        })
    except : 
        pass